<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_digitalTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapy rawling JSON-LD (DigitalTrends)

## Apartado 1.1 Crawler de DigitalTrends extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
DigitalTrends y otros blogs publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org


In [1]:
!pip3 install -U scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.6 MB/s eta 0:00:00


In [2]:
import scrapy
import sys
import json
import locale
import time
import random

import hashlib

from bs4 import BeautifulSoup


# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class DigitalTrendsSpider (scrapy.Spider):
    name = 'DigitalTrends'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.digitaltrends.com']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.digitaltrends.com']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    nDocumentos = 0
    visited_urls = set()

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        if self.nDocumentos >= 1500:
          return

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.b-page '):

            # Cogemos el contenido del título
            title = str (article.css ('.b-headline__title  ').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()

            # autor
            autor = str (article.css ('.author.url.fn ').get ()).strip()
            autor = BeautifulSoup (autor, 'html.parser').get_text().strip()

            # Date
            date = str (article.css ('.b-byline__time.date.dtreviewed').get ()).strip()
            date = BeautifulSoup (date, 'html.parser').get_text().strip()

            #Content
            content = "".join (article.css ('.b-content.b-single__content.h-article-content ').get ())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","").replace("\n","")

            data = {
                'url' : url,
                'title': title,
                'autor': autor,
                'date': date,
                'content': content
            }


            title_hash = hashlib.md5(title.encode()).hexdigest()

            filename = f"{title_hash}.json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                print ("-------------------------")
                print (url)
                print (title)
                print (autor)
                print (date)
                print (content)
                print ("-------------------------")
                self.nDocumentos = self.nDocumentos + 1
                with open ('digitalTrends/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)



        # Obtenemos todas las otros links de la página representados por la etiqueta <a>


        url_in_current_document = response.css ('a')

        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.digitaltrends.com/computing/ o https://www.digitaltrends.com/mobile/'
            # obtenemos el atributo href de la etiqueta <a> y parseamos la página

            url_str = str(next_page.css('::attr(href)').get())


            if ("https://www.digitaltrends.com/computing/" in url_str) or ("/mobile/" in url_str) and url_str not in self.visited_urls and self.nDocumentos < 1500:
              self.visited_urls.add(url_str)
              yield response.follow (next_page, self.parse)

## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

# Nueva sección

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('digitalTrends')):
    os.mkdir('digitalTrends')

# Creamos el proceso con el RSSSpider
process.crawl(DigitalTrendsSpider)
# Ejecutamos el Crawler
process.start()

In [4]:
def contar_archivos_en_carpeta(ruta):
    try:
        # Lista todos los archivos en la carpeta
        archivos = os.listdir(ruta)

        # Filtra solo los archivos (no directorios)
        archivos = [archivo for archivo in archivos if os.path.isfile(os.path.join(ruta, archivo))]

        # Devuelve el número de archivos
        return len(archivos)
    except Exception as e:
        print(f"Error al contar archivos: {e}")
        return None

contar_archivos_en_carpeta("digitalTrends")

1500

Parte 2 – Buscador (0,75 puntos)


In [5]:
!pip install whoosh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 7.4 MB/s eta 0:00:00


In [16]:
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID, DATETIME
from whoosh.qparser import QueryParser
from datetime import datetime
from dateutil import parser
import re

# Define el esquema para el índice de Whoosh
schema = Schema(
    url=ID(unique=True),
    title=TEXT(stored=True),
    author=TEXT(stored=True),
    date=DATETIME(stored=True),
    content=TEXT(stored=True)
)

index_dir = "whooshIndex2"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
    ix = create_in(index_dir, schema)
else:
    ix = open_dir(index_dir)




In [ ]:
# Agrega los documentos al índice
# Aquí deberías leer tus archivos y agregarlos al índice
# Utiliza el bloque 'with' para garantizar que el índice se cierre correctamente
with ix.writer() as writer:
# Recorre todos los archivos en el directorio digitalTrends
  for filename in os.listdir('digitalTrends'):
      if filename.endswith(".json"):  # Asegúrate de que sean archivos JSON
          file_path = os.path.join('digitalTrends', filename)
          with open(file_path, 'r', encoding='utf-8') as file:
              try:
                  file_data = json.load(file)
                  date_string_clean = re.sub(r'\s+\d+:\d+[A|P]M', '', file_data['date'])
                  if date_string_clean != "None" :
                    date_obj = datetime.strptime(date_string_clean, '%B %d, %Y')
                  else :
                    date_obj = None
                  # Agrega los documentos al índice
                  writer.add_document(
                     url=file_data['url'],
                     title=file_data['title'],
                     author=file_data['autor'],
                     date = date_obj,
                     content=file_data['content']
                  )
              except json.JSONDecodeError as e:
                  print(f"Error al procesar el archivo {filename}: {e}")



In [31]:
from whoosh.index import open_dir
from whoosh.qparser import QueryParser
from whoosh import scoring

# Abre el índice creado
ix = open_dir(index_dir)

# Define el parser de consultas
parser = QueryParser("content", schema=ix.schema)

# Ejemplo de búsqueda básica con operadores lógicos
with ix.searcher(weighting=scoring.BM25F) as searcher:
    # Ejemplo de búsqueda con operadores lógicos (AND, OR, NOT)
    query = parser.parse("(Pixel AND Watch) OR Google NOT Apple")

    # Ejecuta la búsqueda
    results = searcher.search(query)

    # Imprime los resultados
    for hit in results:
        print(hit['title'])
        print(hit.highlights("content"))  # Resalta las frases coincidentes


Forget waiting for Google I/O — the Pixel Fold just had a huge leak
channel titled “<b class="match term0">Google</b> <b class="match term1">Pixel</b> x NBA: The Greatest...Furthermore, the <b class="match term0">Google</b> <b class="match term1">Pixel</b> Twitter account shared...bad									Does the <b class="match term0">Google</b> <b class="match term1">Pixel</b> <b class="match term2">Watch</b> work with an iPhone
Forget waiting for Google I/O — the Pixel Fold just had a huge leak
anticipated is the <b class="match term0">Google</b> <b class="match term1">Pixel</b> Fold. And ahead of the...bad									Does the <b class="match term0">Google</b> <b class="match term1">Pixel</b> <b class="match term2">Watch</b> work with an iPhone...I used to love the <b class="match term0">Google</b> <b class="match term1">Pixel</b> Fold. Now, I’m not so sure
There’s a new Pixel Tablet coming in 2023
with the upcoming <b class="match term1">Pixel</b> <b class="match term2">Watch</b>, this new <b cla

In [33]:
# Ejemplo de búsqueda por rango de fechas y orden de resultados
from whoosh.qparser.dateparse import DateParserPlugin

# Añade el plugin para búsqueda por rango de fechas
parser.add_plugin(DateParserPlugin())

with ix.searcher() as searcher:
    # Ejemplo de búsqueda por rango de fechas
    query = parser.parse("date:[2022-01-01 TO 2022-12-31]")

    # Ejecuta la búsqueda con orden de resultados por fecha (ascendente)
    results = searcher.search(query, sortedby="date")

    # Imprime los resultados ordenados por fecha
    for hit in results:
        print(hit['title'], hit['date'])

Scosche unveils several new iPhone MagSafe car mounts at CES 2022 2022-01-03 00:00:00
AT&T and Verizon forge ahead with new 5G deployments despite aviation industry fears 2022-01-03 00:00:00
Scosche unveils several new iPhone MagSafe car mounts at CES 2022 2022-01-03 00:00:00
AT&T and Verizon forge ahead with new 5G deployments despite aviation industry fears 2022-01-03 00:00:00
This wild new display puts a gargantuan 120-inch virtual monitor on your desk 2022-01-04 00:00:00
Microsoft Surface Duo reportedly getting Android 12L despite missing Android 11 2022-01-04 00:00:00
I don’t care if the Asus ROG Flow Z13 is a laptop or a tablet — I just want one 2022-01-04 00:00:00
AT&T’s Fusion 5G is the carrier’s newest affordable 5G smartphone 2022-01-04 00:00:00
How to customize your iPhone backup in iCloud 2022-01-04 00:00:00
TP-Link’s Wi-Fi 6E router comes with motorized antennas for better reception 2022-01-04 00:00:00


In [34]:
from whoosh.qparser import QueryParser
from whoosh.highlight import Highlighter, ContextFragmenter
# Realiza una búsqueda
with ix.searcher() as searcher:
    query = parser.parse("Google")  # Reemplaza 'your_query_here' con tu consulta
    results = searcher.search(query)

    # Configura el fragmenter para resaltar fragmentos de texto
    fragmenter = ContextFragmenter(maxchars=200, surround=30)
    highlighter = Highlighter(fragmenter=fragmenter)

    # Imprime los resultados resaltados
    for hit in results:
        # Obtiene el texto resaltado del campo 'content'
        text = hit.highlights("content", top=2)  # top: Número máximo de fragmentos resaltados por documento

        # Imprime los resultados resaltados
        print(f"Documento: {hit['title']}")
        print(f"Fragmentos resaltados: {text}")


Documento: Which Google One subscription is right for you? Google’s paid plans explained
Fragmentos resaltados: across <b class="match term0">Google</b> Drive, <b class="match term0">Google</b> Photos, and Gmail...Videos		Isn’t <b class="match term0">Google</b> One the same as <b class="match term0">Google</b> Drive?On June 1, <b class="match term0">Google</b> capped unlimited
Documento: Which Google One subscription is right for you? Google’s paid plans explained
Fragmentos resaltados: <b class="match term0">Google</b> One plans have?Sharing your <b class="match term0">Google</b> One plan with family...Videos		Isn’t <b class="match term0">Google</b> One the same as <b class="match term0">Google</b> Drive?On June 1, <b class="match term0">Google</b> capped unlimited
Documento: Heads up — your Google account may get deleted next month
Fragmentos resaltados: surprising. In May, <b class="match term0">Google</b> announced that accounts...the deletion of a <b class="match term0">Google</b>